In [ ]:
from lambeq import CircuitAnsatz
from discopy.quantum.gates import Ry, Rx, Rz, CRx
from discopy.quantum import (
    Bra,
    Circuit,
    CRz,
    Discard,
    H,
    Id,
    Ket,
    qubit,
    Rx, Ry, Rz
)
from discopy.quantum.circuit import Functor
from discopy.grammar.pregroup import Box, Category, Diagram, Ty
from sympy import Symbol


class FslNN(CircuitAnsatz):
    mappings=[]
    def __init__(self, ob_map, n_layers, n_single_qubit_params = 2, discard = False):
        self.preq_embeddings=preq_embeddings
        super().__init__(ob_map,
                         n_layers,
                         n_single_qubit_params,
                         self.circuito,
                         discard,
                         [Rx])
        

                         
    def __call__(self,diagram,maps,width,initial=False):
        return self.circuito(width=width,params=maps,initial=initial)

    def params_shape(self, n_qubits):
        return (self.n_layers + 1, n_qubits)

    def circuito(self,width,params,initial):

        circuit=Ket(*np.zeros(width, dtype=int))

        if initial:
            circuit >>= Id().tensor(*[Ry(Symbol('initialize')) for i in range(width)])
            return circuit

    

        circuit >>= Id().tensor(*[Ry(phase=params[i]) for i in range(width)])
        circuit >>= Id().tensor(*[Rz(phase=params[i]) for i in range(width,2*width)])
        for j in range(width - 1):
            circuit >>= Id(j) @ CRx(phase=params[j+2*width]) @ Id(width - j - 2)


        
        return circuit

In [28]:
import os
import torch
from torch import nn,matmul,kron,bmm
from torch.utils.data import DataLoader
import numpy as np
#from lambeq import BobcatParser, AtomicType, SpacyTokeniser, Rewriter
#from lambeq import IQPAnsatz,Sim15Ansatz
# from utils.FslAnsatzNN import FslSim15Ansatz, FslStronglyEntanglingAnsatz, FslBaseAnsatz
from torchvision import datasets, transforms
import torchvision.transforms as transforms
#from lambeq import NumpyModel,PytorchModel
#from lambeq import IQPAnsatz,Sim15Ansatz
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from icecream import ic

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self,labels,embeddings):
        self.labels=labels
        self.embeddings=embeddings
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self,idx):
        return self.embeddings[idx],self.labels[idx]

In [3]:
preq_embeddings={}
with open("resources\embeddings\common_crawl\glove.42B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "double")
        preq_embeddings[word] = vector

In [4]:
TESTING=True

words=[]
embeddings=[]
for key, value in preq_embeddings.items():
    words.append(key)
    embeddings.append(value)

if TESTING:
    training_words=words[:30]
    training_embeddings=embeddings[:30]

    dev_words=words[31:60]
    dev_embeddings=embeddings[31:60]

# training_embeddings=torch.tensor(  np.array([np.reshape(embedding,(1,300)) for embedding in training_embeddings]) ,requires_grad=True )
# dev_embeddings=torch.tensor(np.array([np.reshape(embedding,(1,300)) for embedding in dev_embeddings]),requires_grad=True)

training_embeddings=[np.reshape(embedding,(1,300)) for embedding in training_embeddings]
dev_embeddings=[np.reshape(embedding,(1,300)) for embedding in dev_embeddings]

In [5]:
def create_data(embeddings):
    training_data=[]
    training_labels=[]
    for i in range(len(embeddings)):
        for j in range(len(embeddings)):

            training_labels.append(np.inner(embeddings[i],embeddings[j])[0][0])

            new_embedding=np.append(embeddings[i],embeddings[j])

            training_data.append(new_embedding)

    training_data=torch.tensor(training_data,requires_grad=True)

    training_labels=torch.tensor(training_labels,requires_grad=True)

    return training_data, training_labels


In [112]:
y_gate=torch.tensor([[0,-1j],[1j,0]],dtype=torch.complex128,requires_grad=True)
x_gate=torch.tensor([[0,1],[1,0]],dtype=torch.complex128,requires_grad=True)
z_gate=torch.tensor([[1,0],[0,-1]],dtype=torch.complex128,requires_grad=True)

def Id():
    gate=torch.tensor([[1.+0j,0],[0,1.+0j]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def CRx(theta):
    gate=torch.tensor( [ [1.,0,0,0], [0, 1. ,0,0] ,[0,0,torch.cos(theta/2), -1j*torch.sin(theta/2) ] , [0,0,-1j*torch.sin(theta/2) , torch.cos(theta/2)] ]   ,requires_grad=True)   
    gate.retain_grad()
    return gate

def zero_bra():
    gate=torch.tensor([1.+0j,0,0,0,0,0,0,0],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def zero_ket():
    gate=torch.tensor([[1.+0j],[0],[0],[0],[0],[0],[0],[0]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def zero_1d_ket():
    gate=torch.tensor([[1.+0j],[0]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def zero_1d_bra():
    gate=torch.tensor([[1.+0j,0]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def one_1d_ket():
    gate=torch.tensor([[0],[1.+0j]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def one_1d_bra():
    gate=torch.tensor([[0,1.+0j]],requires_grad=True,dtype=torch.complex128)
    gate.retain_grad()
    return gate

def Ry(theta):
    gate = torch.linalg.matrix_exp(-0.5j*theta[:,:,None]*y_gate)
    return gate

def Rx(theta):
    gate = torch.linalg.matrix_exp(-0.5j*theta[:,:,None]*x_gate)
    return gate

def Rx(theta):
    gate = torch.linalg.matrix_exp(-0.5j*theta[:,:,None]*z_gate)
    return gate

#Rx=lambda x: torch.exp(-0.5j*x*torch.tensor([[0,1],[1,0]],dtype=torch.complex128))

#Ry=lambda x: torch.exp(-0.5j*x*torch.tensor([[0,-1j],[1j,0]],dtype=torch.complex128))

CRx=lambda x: kron(Id(),matmul(zero_1d_ket(),zero_1d_bra()))+kron(Rx(x),matmul(one_1d_ket(),one_1d_bra()))

In [ ]:
s=torch.tensor([[1.,2,3,4],[1.,2,3,4],[1.,2,3,4]],requires_grad=True)
n=torch.tensor([[0,1]  ,[1.,0]],requires_grad=True)
s2=torch.reshape(s,(3,4,1))
s3=s2[:,:,None]*n
print(s3.shape)
torch.linalg.matrix_exp(1j*0.5*np.pi*s3).shape

In [7]:
training_data,training_labels=create_data(training_embeddings)
dev_data,dev_labels=create_data(dev_embeddings)

C:\Users\ucl_l\AppData\Local\Temp\ipykernel_20716\535721322.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  training_data=torch.tensor(training_data,requires_grad=True)


In [8]:
training_object=CustomDataset(training_labels,training_data)
dev_object=CustomDataset(dev_labels,dev_data)

training_loader=DataLoader(training_object,batch_size=15)
validation_loader=DataLoader(dev_object,batch_size=15)

## Model and training setting

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [10]:
#loss_fn=lambda x,y:np.abs(x-y)
# def loss_fn(output,target):
#     print(output,target)
#     return nn.MSELoss(output,target)
loss_fn = torch.nn.MSELoss()

In [115]:
N_PARAMS=8
class PreQ(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=0)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(300, 200),
            nn.ReLU(),
            # nn.Linear(300, 250),
            # nn.ReLU(),
            # nn.Linear(250, 200),
            #nn.ReLU(),
            nn.Linear(200, 50),
            nn.ReLU(),
            nn.Linear(50,N_PARAMS)
        )    
        self.double()

    def forward(self, x):
        #x = self.flatten(x)
        x.requires_grad_()
        x.retain_grad()
        logits1=self.linear_relu_stack(x[:,0:300])
        logits2=self.linear_relu_stack(x[:,300:600])

        logits1_reshaped=torch.reshape(logits1,(logits1.shape[0],logits1.shape[1],1))
        logits2_reshaped=torch.reshape(logits2,(logits2.shape[0],logits2.shape[1],1))
        
        bra=torch.stack([zero_bra()[None] for i in range(logits1_reshaped.shape[0])])
        ket=torch.stack([zero_ket() for i in range(logits1_reshaped.shape[0])])

        circuit1=bmm(bra,self.get_quantum_state(parameters=logits2_reshaped).mH)
        circuit2=bmm(self.get_quantum_state(parameters=logits1_reshaped),ket)

        output=self.flatten(bmm(circuit1,circuit2))

        return output

    def get_quantum_state(self,parameters):
        first_layer=torch.stack( [   kron(kron( Rx(parameters[:,0])[i],Rx(parameters[:,1])[i] ),Rx(parameters[:,2])[i] )  for i in range(parameters.shape[0])   ]   )
        second_layer=torch.stack( [   kron(kron( Ry(parameters[:,0])[i],Ry(parameters[:,1])[i] ),Ry(parameters[:,2])[i] )  for i in range(parameters.shape[0])   ]  )
        
        third_layer=kron(CRx(parameters[:,6]),Id())
        fourth_layer=kron(Id(),CRx(parameters[:,7]))
        
        output=bmm(bmm(bmm(first_layer,second_layer),third_layer),fourth_layer)
        return output

model = PreQ().to(device)
        

In [91]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [116]:
optimizer.zero_grad()

x = torch.randn(3,600,requires_grad = True,dtype=torch.double)
# x=training_data
y = model(x)

y=y.type(torch.DoubleTensor)

z=torch.randn(y.shape,requires_grad=True,dtype=torch.double)
z.retain_grad()

loss=loss_fn(y,z)
loss.backward()
ic(loss.grad)

optimizer.step()


# for name, param in model.named_parameters():
#     print(name, param.grad)

ic| logits1: tensor([[-0.1297, -0.1242, -0.1167,  0.0371, -0.0249, -0.0699, -0.1014,  0.1559],
                     [-0.2436, -0.1266,  0.0152,  0.0252, -0.0543, -0.0907, -0.2159,  0.0193],
                     [-0.1705, -0.1355, -0.0463, -0.1054, -0.0694, -0.1704, -0.0896,  0.0599]],
                    dtype=torch.float64, grad_fn=<AddmmBackward0>)
C:\Users\ucl_l\AppData\Local\Temp\ipykernel_20716\118236819.py:14: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  ic(loss.grad)
ic| loss.grad: None


In [99]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data


        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        
        #ic(inputs,labels)
        output = model(inputs)

        output=output.to(dtype=torch.double)


        # Compute the loss and its gradients
        loss = loss_fn(output, labels)
        ic(loss.grad)
        loss.backward()
        

        # for name, param in model.named_parameters():
        #     print(name, param.grad)

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 15 == 14:
            last_loss = running_loss / 15 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [63]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/Prueba{}'.format(timestamp))
epoch_number = 0
EPOCHS = 1
best_vloss = 1_000_000.

In [64]:
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata

            voutputs = model(vinputs)
            voutputs=voutputs.to(dtype=torch.double)

            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS-- Train: {} Valid: {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'runs/NN/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


C:\Users\ucl_l\AppData\Local\Temp\ipykernel_20716\1488263493.py:23: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  ic(loss.grad)
ic| loss.grad: None


EPOCH 1:


ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None
ic| loss.grad: None


  batch 15 loss: 1528.4703182625672


KeyboardInterrupt: 

In [37]:
for name, param in model.named_parameters():
    print(name, param.grad)

linear_relu_stack.0.weight tensor([[ 6.8447e-04, -2.5461e-04,  1.2643e-05,  ...,  6.6182e-04,
         -5.8039e-04, -7.9825e-04],
        [-2.9097e-04,  3.8870e-04,  1.9614e-04,  ..., -2.6235e-04,
          4.4786e-04,  9.1579e-04],
        [-9.5257e-04,  7.4502e-04,  2.2726e-04,  ..., -4.2169e-04,
          6.8613e-04,  1.3927e-03],
        ...,
        [ 7.8595e-05, -6.7144e-06, -1.5655e-04,  ...,  1.2636e-05,
         -8.1359e-05,  8.7035e-05],
        [-3.0178e-05,  1.8431e-04,  1.6299e-04,  ..., -6.3107e-05,
          1.6870e-04,  2.5272e-04],
        [ 1.8042e-05,  3.0062e-05, -2.2960e-05,  ..., -4.1022e-05,
         -3.2452e-05,  2.8035e-05]], dtype=torch.float64)
linear_relu_stack.0.bias tensor([-2.4744e-03, -2.7003e-05,  7.1029e-04, -1.5306e-03,  7.4028e-05,
         0.0000e+00, -1.9793e-03, -2.0840e-03,  4.4033e-04,  1.7426e-03,
         2.5941e-04,  5.8927e-04, -8.0517e-05,  7.3306e-05, -3.0759e-04,
         0.0000e+00, -4.3708e-04,  3.4158e-03, -1.4465e-03, -9.2754e-04,
   